In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt, gridspec
from plotutils import *

In [ ]:
basedir = '../benchmarks_py/sfi_scaling006'

DO_SAVE            = True # whether to save pdf output


In [ ]:
scalingtype = 'agents'
COL, desc, data, mutators = get_data(basedir, scalingtype)

mutator_scaling_slopes = {}
for mndx, m in enumerate(sorted(mutators)):
    _, xs, ys, _ = get_scaling_vals(data, COL, m, 'Comprehension')
    slope,_ = plot_scatter(xs, ys, loglog=True, regression_line=True, do_plot=False)
    mutator_scaling_slopes[m] = slope

for pCol in ['Comprehension']: # 'Comprehension', 'AcceptanceRate', 'MeanGrammarDrift', 'AgentGrammarDrift', 'GrammarVar']:
    print("Doing %s" % pCol)
    plt.figure(figsize=(15,4*len(mutators)))
    plt.suptitle('Scaling with %s'%desc, fontsize=17, y=1.01)
    
    gs = gridspec.GridSpec(len(mutators), 3)
    for mndx, m in enumerate(sorted(mutators)):
        plot_dfs, xs, ys, ylabel = get_scaling_vals(data, COL, m, pCol)
        
        plt.subplot(gs[mndx,0])
        for cfg, df in plot_dfs:
            plt.plot(df.Step, df[pCol], label=cfg[COL])
        plt.xlabel('Step')
        plt.ylabel(pCol)
        plt.semilogx()
        plt.title(m + '-'+pCol, fontsize=13)
        
        plt.subplot(gs[mndx,1])
        if len(xs):
            slope, _ = plot_scatter(xs, ys, loglog=True, regression_line=True)

        plt.xlabel(desc)
        plt.ylabel(ylabel)

        plt.subplot(gs[mndx,2])
        plt.semilogx()
        
        ylabel = pCol
        for cfg, df in plot_dfs:
            v = float(cfg[COL])
            vals = df[pCol]
            rescaled = df.Step/(v**mutator_scaling_slopes[m])
            show_line = True
            if pCol.endswith('Drift') or pCol.endswith('GrammarVar'):
                ylabel = '$%s \\times %s^{%0.2f}$' % (pCol, desc.replace('#', '\\#'), -slope)
                vals = vals/(v**slope)
            l=plt.plot(rescaled, vals, label=cfg[COL])
            if not show_line: l[0].set_alpha(0.0)
            if v==21:
                plt.xlim([0, 2*np.max(rescaled)])
        plt.xticks([])
        plt.xlabel('$Steps \\times %s^{%0.2f}$' % (desc.replace('#', '\\#'), -mutator_scaling_slopes[m]))
        plt.ylabel(ylabel)
        leg = plt.legend(title=desc)
        for lh in leg.legendHandles: 
            lh.set_alpha(1)
    
    plt.tight_layout()
    
    if DO_SAVE:
        plt.savefig('output/scaling_%s_%s.pdf'%(scalingtype, pCol), bbox_inches='tight')
    